In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# from keras.wrappers import KerasClassifier

import matplotlib.pyplot as plt
from matplotlib import gridspec

ModuleNotFoundError: No module named 'tensorflow.keras.wrappers'

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
ds_train = image_dataset_from_directory(
    '../cifar-10-stable-diffusion-detection/train',
    labels='inferred',
    label_mode='binary',
    image_size=[64,64],
    interpolation='nearest',
    batch_size=32,
    shuffle=True,
)

In [ ]:
ds_test = image_dataset_from_directory(
    '../cifar-10-stable-diffusion-detection/test',
    labels='inferred',
    label_mode='binary',
    image_size=[64,64],
    interpolation='nearest',
    batch_size=32,
    shuffle=True,
)

In [ ]:
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE


In [ ]:
ds_train = (
    ds_train
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)


In [ ]:
ds_valid = (
    ds_test
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

In [ ]:
def create_model():
    model = Sequential([
        layers.InputLayer(input_shape=[64, 64, 3]),
        preprocessing.RandomFlip(mode='horizontal'),
        preprocessing.RandomRotation(factor=0.05), 
        layers.BatchNormalization(renorm=True),
        layers.Conv2D(filters=8, kernel_size=3, activation='relu', padding='same'),
        layers.MaxPool2D(),
        layers.BatchNormalization(renorm=True),
        layers.Conv2D(filters=8, kernel_size=3, activation='relu', padding='same'),
        layers.MaxPool2D(),
        layers.BatchNormalization(renorm=True),
        layers.Conv2D(filters=8, kernel_size=3, activation='relu', padding='same'),
        layers.Conv2D(filters=8, kernel_size=3, activation='relu', padding='same'),
        layers.MaxPool2D(),
        layers.BatchNormalization(renorm=True),
        layers.Flatten(),
        layers.Dense(32, activation='relu'),
        layers.Dropout(0.1),  
        layers.Dense(1, activation='sigmoid'),
    ])
    optimizer = tf.keras.optimizers.Adam(epsilon=0.01)
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['binary_accuracy'],
    )
    return model


In [ ]:
keras_model = keras.wrappers.scikit_learn.KerasClassifier(
    build_fn=create_model,
    epochs=50,
    verbose=0
)


In [ ]:
param_grid = {
    'batch_size': [16, 32, 64],
    'learning_rate': [0.001, 0.01, 0.1],
    'dropout_rate': [0.0, 0.1, 0.2],
}


In [ ]:
grid_search = GridSearchCV(estimator=keras_model, param_grid=param_grid, cv=3)
grid_result = grid_search.fit(ds_train)

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=5,
    min_delta=0.001,
    restore_best_weights=True,
)

In [ ]:
optimizer = tf.keras.optimizers.Adam(epsilon=0.01)
model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=50,
    callbacks=[early_stopping],
)

# Plot learning curves
import pandas as pd
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot();
